<a href="https://colab.research.google.com/github/japmansingh/DS_CAPSTONE_IBM/blob/master/BusinessProspectsFinalCompleted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Capstone Project
### This project is aimed at businessmen who are planning to open a business in British Columbia, Canada. It takes into account the population density of the region, the frequency of various businesses in the locality. 

In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

## Let's start by importing the necessary libraries

In [5]:
!pip install geopandas
!pip install geocoder
#!pip install googlemaps
!pip install opencage

     |████████████████████████████████| 931kB 3.1MB/s 
     |████████████████████████████████| 10.9MB 47.5MB/s 
     |████████████████████████████████| 14.7MB 315kB/s 
     |████████████████████████████████| 102kB 2.4MB/s 
     |████████████████████████████████| 61kB 2.0MB/s 
     |████████████████████████████████| 2.7MB 8.5MB/s 


In [0]:
import requests
import pandas as pd
import numpy as np
import lxml.html as lh
import folium
import geocoder
import geopy
import geopandas
import matplotlib.pyplot as plt
#from googlemaps import Client as GoogleMaps
from opencage.geocoder import OpenCageGeocode

In [0]:
pd.options.display.max_seq_items = 2000

### Source is added for the Pin Codes and related data and is stored in the 'url' variable.

In [0]:
url=requests.get(
    'https://www.geonames.org/postal-codes/CA/BC/british-columbia.html').text
doc = lh.fromstring(url)
tr_elements = doc.xpath('//tr')

### BeautifulSoup is used to scrape data from the given url.

In [0]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(url,'lxml')
#print(soup.prettify())

In [0]:
table=soup.find('table',{ 'class': "restable"})
#print(table.prettify())

In [0]:
postalCode = []
placeName = []
neighbourhood = []

In [12]:
[len(T) for T in tr_elements[:7]]

[2, 3, 7, 7, 2, 7, 2]

### An attempt is made to scrape the data but due to unequal dimensions, it was out of the boundaries for this course and project.

In [0]:
#for row in soup.find('table').find_all('tr'):
 #   cells=row.find_all('td')
  #  if(len(cells)>0):
   #     postalCode.append(cells[0].text.rstrip('\n'))
    #    placeName.append(cells[1].text.rstrip('\n'))
     #   neighbourhood.append(cells[2].text.rstrip('\n'))

### An excel file is hence loaded from the local system and used for further analysis.

In [14]:
post_df=pd.read_csv("https://raw.githubusercontent.com/japmansingh/DS_CAPSTONE_IBM/master/PostalPopulation.CSV", header=None, error_bad_lines=False)
post_df.columns=['Locality','Code','LocalPopulation','Area','AreaPopulation']
post_df.head()


,Locality,Code,LocalPopulation,Area,AreaPopulation
0,Cariboo and West Okanagan (100 Mile House),V0K,32908,100 Mile House,32908
1,Abbotsford Southeast,V2S,48647,Abbotsford,141682
2,Abbotsford Southwest,V2T,56111,Abbotsford,141682
3,Abbotsford East,V3G,24150,Abbotsford,141682
4,Abbotsford West,V4X,12774,Abbotsford,141682


In [15]:
addresses = post_df.drop(['LocalPopulation','Area','AreaPopulation'], axis=1)
addresses.head()

,Locality,Code
0,Cariboo and West Okanagan (100 Mile House),V0K
1,Abbotsford Southeast,V2S
2,Abbotsford Southwest,V2T
3,Abbotsford East,V3G
4,Abbotsford West,V4X


In [16]:
key = 'b1c8becdc3944a44830d56b947d608f6'
geocoder = OpenCageGeocode(key)
query = 'Southeast Abbotsford'  
#lat': "49° 3' 7.61832'' N", 'lng': "122° 19' 46.12512'' W"
#lat': "49° 5' 9.96000'' N", 'lng': "122° 24' 9.36000'' W"
results = geocoder.geocode(query)
print (results)
lat = results[0]['geometry']['lat']
long = results[0]['geometry']['lng']
print (lat, long)

[{'annotations': {'DMS': {'lat': "50° 58' 22.30320'' N", 'lng': "0° 8' 40.58520'' E"}, 'MGRS': '30UYB0046150688', 'Maidenhead': 'IO90wx23pl', 'Mercator': {'x': -16097.578, 'y': 6583280.358}, 'OSM': {'edit_url': 'https://www.openstreetmap.org/edit?node=829641671#map=17/50.97286/-0.14461', 'note_url': 'https://www.openstreetmap.org/note/new#map=17/50.97286/-0.14461&layers=N', 'url': 'https://www.openstreetmap.org/?mlat=50.97286&mlon=-0.14461#map=17/50.97286/-0.14461'}, 'UN_M49': {'regions': {'EUROPE': '150', 'GB': '826', 'NORTHERN_EUROPE': '154', 'WORLD': '001'}, 'statistical_groupings': ['MEDC']}, 'callingcode': 44, 'currency': {'alternate_symbols': [], 'decimal_mark': '.', 'html_entity': '&#x00A3;', 'iso_code': 'GBP', 'iso_numeric': '826', 'name': 'British Pound', 'smallest_denomination': 1, 'subunit': 'Penny', 'subunit_to_unit': 100, 'symbol': '£', 'symbol_first': 1, 'thousands_separator': ','}, 'flag': '🇬🇧', 'geohash': 'gcpcuxqdqqjjwt492yqp', 'qibla': 118.47, 'roadinfo': {'drive_on':

In [0]:
list_lat = []   # create empty lists
list_long = []
for index, row in addresses.iterrows(): # iterate over rows in dataframe

    Locality = row['Locality']
    State = 'British Columbia'
    Code = row['Code']       
    query = str(Code)+' '+str(State)

    results = geocoder.geocode(query)
    try:
        lat = results[0]['geometry']['lat']
        long = results[0]['geometry']['lng']

        list_lat.append(lat)
        list_long.append(long)
    except:
        list_lat.append('Null')
        list_long.append('Null')


In [0]:
addresses['latitude'] = list_lat
addresses['longitude'] = list_long

In [19]:
addresses.head(100)

,Locality,Code,latitude,longitude
0,Cariboo and West Okanagan (100 Mile House),V0K,51.524836,-0.026248
1,Abbotsford Southeast,V2S,49.044405,-122.297609
2,Abbotsford Southwest,V2T,49.035765,-122.360287
3,Abbotsford East,V3G,53.999830,-125.003200
4,Abbotsford West,V4X,53.999830,-125.003200
...,...,...,...,...
95,Qualicum Beach,V9K,51.524836,-0.026248
96,Inside Passage and the Queen Charlottes (Queen...,V0T,53.999830,-125.003200
97,Quesnel,V2J,53.999830,-125.003200
98,Central Okanagan and High Country (Revelstoke),V0E,49.204818,-122.906133


In [20]:
"""for x in range(len(addresses)):
    geocode_result = gmaps.geocode(addresses['FullAddress'][x])
    addresses['lat'][x] = geocode_result[0]['geometry']['location'] ['lat']
    addresses['long'][x] = geocode_result[0]['geometry']['location']['lng']
addresses.head()"""

"for x in range(len(addresses)):\n    geocode_result = gmaps.geocode(addresses['FullAddress'][x])\n    addresses['lat'][x] = geocode_result[0]['geometry']['location'] ['lat']\n    addresses['long'][x] = geocode_result[0]['geometry']['location']['lng']\naddresses.head()"

In [0]:
gdf = geopandas.GeoDataFrame(post_df, geometry=geopandas.points_from_xy(addresses.longitude, addresses.latitude))

In [22]:
gdf.head()

,Locality,Code,LocalPopulation,Area,AreaPopulation,geometry
0,Cariboo and West Okanagan (100 Mile House),V0K,32908,100 Mile House,32908,POINT (-0.02625 51.52484)
1,Abbotsford Southeast,V2S,48647,Abbotsford,141682,POINT (-122.29761 49.04441)
2,Abbotsford Southwest,V2T,56111,Abbotsford,141682,POINT (-122.36029 49.03577)
3,Abbotsford East,V3G,24150,Abbotsford,141682,POINT (-125.00320 53.99983)
4,Abbotsford West,V4X,12774,Abbotsford,141682,POINT (-125.00320 53.99983)


In [23]:
# define the world map
world_map = folium.Map(location=[55.130, -116.35], zoom_start=6)
# display world map
world_map

In [0]:
for index, row in addresses.iterrows():
    folium.CircleMarker([row['latitude'], row['longitude']],
                        radius=15,
                        popup=row['Locality'],
                        fill_color="#3db7e4", # divvy color
                        control_scale=True,
                       ).add_to(world_map)

In [25]:
world_map

In [0]:
import folium

In [27]:
CLIENT_ID = 'WIWBJZZ1DDKOYYRHS1KCCPTXG3YDGGGV3XTMBZ1DA3WSXPOL'
CLIENT_SECRET = 'KGWXLHCLOJBGK4BTM551Z1UIN5ZUWLX0O3OLFKHIKKKE1RN5'
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WIWBJZZ1DDKOYYRHS1KCCPTXG3YDGGGV3XTMBZ1DA3WSXPOL
CLIENT_SECRET:KGWXLHCLOJBGK4BTM551Z1UIN5ZUWLX0O3OLFKHIKKKE1RN5


In [28]:
'''def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)'''

'def getNearbyVenues(names, latitudes, longitudes, radius=500):\n    \n    venues_list=[]\n    for name, lat, lng in zip(names, latitudes, longitudes):\n        print(name)\n            \n        # create the API request URL\n        url = \'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}\'.format(\n            CLIENT_ID, \n            CLIENT_SECRET, \n            VERSION, \n            lat, \n            lng, \n            radius, \n            LIMIT)\n            \n        # make the GET request\n        results = requests.get(url).json()["response"][\'groups\'][0][\'items\']\n        \n        # return only relevant information for each nearby venue\n        venues_list.append([(\n            name, \n            lat, \n            lng, \n            v[\'venue\'][\'name\'], \n            v[\'venue\'][\'location\'][\'lat\'], \n            v[\'venue\'][\'location\'][\'lng\'],  \n            v[\'venue\'][\'categories\'][0][\'n

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=10000):
    
    venues_list=[]

    for name, lat, lng in zip(names, latitudes, longitudes):
        try:
            print(name)
            
            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
            results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
            venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
        except:
            continue

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [30]:
venues = getNearbyVenues(names=addresses['Locality'],
                                   latitudes=addresses['latitude'],
                                   longitudes=addresses['longitude']
                                  )

Cariboo and West Okanagan (100 Mile House)
Abbotsford Southeast
Abbotsford Southwest
Abbotsford East
Abbotsford West
Harrison Lake Region (Agassiz)
Chilcotin (Alexis Creek)
Atlin Region (Atlin)
Burnaby (East Big Bend / Stride Avenue / Edmonds / Cariboo-Armstrong)
Burnaby (Burnaby Heights / Willingdon Heights / West Central Valley)
Burnaby (Lakeview-Mayfield / Richmond Park / Kingsway-Beresford)
Burnaby (Cascade-Schou / Douglas-Gilpin)
Burnaby (Maywood / Marlborough / Oakalla / Windsor)
Burnaby (Suncrest / Sussex-Nelson / Clinton-Glenwood / West Big Bend)
Burnaby (Government Road / Lake City / SFU / Burnaby Mountain)
Burnaby (Parkcrest-Aubrey / Ardingley-Sprott)
Campbell River Central
Campbell River Outskirts
Castlegar
Cedar
Central Island (Chemainus)
Chilliwack Central
Chilliwack West
Chilliwack East
Comox
Coquitlam North
Coquitlam South
Courtenay Northern Outskirts
Courtenay Central
Cranbrook
Dawson Creek
Delta Northeast
Delta East
Delta East Central
Delta Central
Delta Southeast
Delt

In [31]:
venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Cariboo and West Okanagan (100 Mile House),51.524836,-0.026248,The Lord Tredegar,51.528390,-0.033485,Pub
1,Cariboo and West Okanagan (100 Mile House),51.524836,-0.026248,The Morgan Arms,51.527908,-0.030935,Bar
2,Cariboo and West Okanagan (100 Mile House),51.524836,-0.026248,The Coffee Room,51.525598,-0.034857,Coffee Shop
3,Cariboo and West Okanagan (100 Mile House),51.524836,-0.026248,Mile End Lock,51.524492,-0.036972,Canal Lock
4,Cariboo and West Okanagan (100 Mile House),51.524836,-0.026248,Greedy Cow,51.525283,-0.034823,Burger Joint
...,...,...,...,...,...,...,...
2254,White Rock,33.354893,-111.469028,Rosati's Pizza,33.361316,-111.470724,Pizza Place
2255,White Rock,33.354893,-111.469028,Shell,33.358597,-111.467381,Gas Station
2256,White Rock,33.354893,-111.469028,McDonald's,33.358539,-111.467266,Fast Food Restaurant
2257,White Rock,33.354893,-111.469028,Roberts Resorts - Gold Canyon RV Resort,33.354896,-111.471759,RV Park


In [32]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Abbotsford Southeast,27,27,27,27,27,27
Abbotsford Southwest,4,4,4,4,4,4
Burnaby (Burnaby Heights / Willingdon Heights / West Central Valley),30,30,30,30,30,30
Burnaby (East Big Bend / Stride Avenue / Edmonds / Cariboo-Armstrong),30,30,30,30,30,30
Burnaby (Lakeview-Mayfield / Richmond Park / Kingsway-Beresford),5,5,5,5,5,5
...,...,...,...,...,...,...
Vernon East,5,5,5,5,5,5
West Kootenays (Rossland),30,30,30,30,30,30
West Vancouver North,5,5,5,5,5,5


In [33]:
print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))

There are 230 uniques categories.


In [0]:
# one hot encoding
venues_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
venues_onehot['Neighborhood'] = venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])
venues_onehot = venues_onehot[fixed_columns]

In [35]:
venues_onehot.head()

,Neighborhood,African Restaurant,American Restaurant,Amphitheater,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Garden,Belgian Restaurant,Bike Rental / Bike Share,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Bus Station,Bus Stop,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Canal Lock,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,...,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Stadium,Steakhouse,Storage Facility,Supermarket,Surf Spot,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Water Park,Waterfront,Wine Shop,Women's Store,Yoga Studio
0,Cariboo and West Okanagan (100 Mile House),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Cariboo and West Okanagan (100 Mile House),0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Cariboo and West Okanagan (100 Mile House),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Cariboo and West Okanagan (100 Mile House),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Cariboo and West Okanagan (100 Mile House),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [36]:
venues_grouped = venues_onehot.groupby('Neighborhood').mean().reset_index()
venues_grouped

,Neighborhood,African Restaurant,American Restaurant,Amphitheater,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Garden,Belgian Restaurant,Bike Rental / Bike Share,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Bus Station,Bus Stop,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Canal Lock,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,...,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Stadium,Steakhouse,Storage Facility,Supermarket,Surf Spot,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Water Park,Waterfront,Wine Shop,Women's Store,Yoga Studio
0,Abbotsford Southeast,0.0,0.000000,0.0,0.0,0.0,0.037037,0.0,0.0,0.0,0.00,0.000000,0.0,0.037037,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.037037,0.000000,0.0,0.000000,0.037037,0.037037,0.0,0.0,0.037037,0.0,0.0,0.0,0.0,0.0,0.037037,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.037037,0.0,0.0,0.0,0.000000,0.000000,0.074074,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.037037,0.0,0.0,0.000000,0.0,0.0
1,Abbotsford Southwest,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.25,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
2,Burnaby (Burnaby Heights / Willingdon Heights ...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.000000,0.0,0.033333,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.033333,0.0,0.000000,0.033333,0.000000,0.0,0.0,0.100000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.033333,0.000000,0.066667,0.0,0.0,0.033333,0.0,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.066667,0.0,0.0,0.000000,0.0,0.0
3,Burnaby (East Big Bend / Stride Avenue / Edmon...,0.0,0.033333,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.033333,0.0,0.033333,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,0.000000,0.0,0.000000,0.033333,0.000000,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.0,0.033333,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.033333,0.000000,0.000000,0.0,0.0,0.033333,0.0,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
4,Burnaby (Lakeview-Mayfield / Richmond Park / K...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,Vernon East,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0000

In [37]:
venues_grouped.shape

(113, 231)

In [38]:
num_top_venues = 5

for hood in venues_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = venues_grouped[venues_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Abbotsford Southeast----
               venue  freq
0     Sandwich Place  0.11
1        Coffee Shop  0.11
2   Sushi Restaurant  0.07
3        Pizza Place  0.04
4  Convenience Store  0.04


----Abbotsford Southwest----
                venue  freq
0     Automotive Shop  0.25
1        Home Service  0.25
2            Pharmacy  0.25
3   Other Repair Shop  0.25
4  African Restaurant  0.00


----Burnaby (Burnaby Heights / Willingdon Heights / West Central Valley)----
                   venue  freq
0            Coffee Shop  0.10
1                   Café  0.10
2       Sushi Restaurant  0.07
3      French Restaurant  0.07
4  Vietnamese Restaurant  0.07


----Burnaby (East Big Bend / Stride Avenue / Edmonds / Cariboo-Armstrong)----
                venue  freq
0         Comedy Club  0.07
1         Coffee Shop  0.07
2              Market  0.03
3           Multiplex  0.03
4  Chinese Restaurant  0.03


----Burnaby (Lakeview-Mayfield / Richmond Park / Kingsway-Beresford)----
                  venu

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [40]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = venues_grouped['Neighborhood']

for ind in np.arange(venues_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(venues_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abbotsford Southeast,Sandwich Place,Coffee Shop,Sushi Restaurant,Bus Station,Convenience Store,Café,Middle Eastern Restaurant,Chinese Restaurant,Burger Joint,Sporting Goods Shop
1,Abbotsford Southwest,Home Service,Pharmacy,Automotive Shop,Other Repair Shop,Yoga Studio,Field,Fried Chicken Joint,French Restaurant,Food Truck,Food Court
2,Burnaby (Burnaby Heights / Willingdon Heights ...,Café,Coffee Shop,French Restaurant,Vietnamese Restaurant,Sushi Restaurant,Thai Restaurant,Bakery,Fried Chicken Joint,Burger Joint,Sporting Goods Shop
3,Burnaby (East Big Bend / Stride Avenue / Edmon...,Comedy Club,Coffee Shop,Market,Fast Food Restaurant,Supermarket,Multiplex,Breakfast Spot,Restaurant,Burger Joint,Rock Club
4,Burnaby (Lakeview-Mayfield / Richmond Park / K...,RV Park,Gas Station,Pizza Place,Fast Food Restaurant,Farmers Market,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food


In [41]:
ind_rest=venues_grouped[['Neighborhood','Indian Restaurant']]
ind_rest.sort_values(by=['Indian Restaurant'], inplace=True, ascending=False)
ind_rest

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Neighborhood,Indian Restaurant
93,Vancouver (SE Oakridge / East Marpole / South ...,0.125000
47,North Vancouver Southwest Central,0.066667
69,South Okanagan (Summerland),0.043478
91,Vancouver (SE Kensington / Victoria-Fraserview),0.035714
86,Vancouver (NW Shaughnessy / East Kitsilano / Q...,0.033333
...,...,...
33,Maple Ridge Northwest,0.000000
32,Langley Township Northwest,0.000000
31,Langley City,0.000000
30,Kitimat,0.000000


In [0]:
population=addresses.drop(['Code'],axis=1)

ind_rest.rename(columns={"Neighborhood": "Locality","Indian Restaurant":"IndianRestaurant"},inplace=True)

pd.options.mode.chained_assignment = None  # default='warn'

finaldf = pd.merge(population, ind_rest, on='Locality')

In [57]:
finaldf.sort_values(by=['IndianRestaurant'], inplace=True, ascending=False)
finaldf

,Locality,latitude,longitude,IndianRestaurant
83,Vancouver (SE Oakridge / East Marpole / South ...,49.215172,-123.097003,0.125000
101,North Vancouver Southwest Central,49.319186,-123.083103,0.066667
67,South Okanagan (Summerland),49.028667,-119.458294,0.043478
77,Vancouver (SE Kensington / Victoria-Fraserview),49.222370,-123.068315,0.035714
84,Vancouver (West Mount Pleasant / West Riley Pa...,49.254759,-123.105073,0.033333
...,...,...,...,...
34,Merritt,51.524836,-0.026248,0.000000
33,Maple Ridge Northwest,53.911321,-122.783791,0.000000
32,Langley City,49.101176,-122.673733,0.000000
31,Langley Township Northwest,48.424523,-123.363029,0.000000


In [0]:
businessmap = folium.Map(location=[55.130, -116.35], zoom_start=6)

In [45]:
'''
for index, row in addresses.iterrows():
    freq = finaldf['Indian Restaurant']
    if freq == 0.000000:
        folium.CircleMarker([row['latitude'], row['longitude']],
                        radius=15,
                        popup=row['Locality'],
                        fill_color="#3db7e4", # divvy color
                        control_scale=True,
                       ).add_to(businessmap)
        print('done')
    else:
        continue
'''

'\nfor index, row in addresses.iterrows():\n    freq = finaldf[\'Indian Restaurant\']\n    if freq == 0.000000:\n        folium.CircleMarker([row[\'latitude\'], row[\'longitude\']],\n                        radius=15,\n                        popup=row[\'Locality\'],\n                        fill_color="#3db7e4", # divvy color\n                        control_scale=True,\n                       ).add_to(businessmap)\n        print(\'done\')\n    else:\n        continue\n'

In [0]:
finalplot = finaldf[finaldf.IndianRestaurant == 0.000000]

In [60]:
finalplot

,Locality,latitude,longitude,IndianRestaurant
63,Sidney,33.354893,-111.469028,0.0
59,West Kootenays (Rossland),49.204818,-122.906133,0.0
81,Vancouver (West Kensington / NE Riley Park-Lit...,49.247634,-123.090621,0.0
80,Vancouver (East Mount Pleasant),49.261967,-123.091194,0.0
79,Vancouver (Killarney),49.215677,-123.041565,0.0
...,...,...,...,...
34,Merritt,51.524836,-0.026248,0.0
33,Maple Ridge Northwest,53.911321,-122.783791,0.0
32,Langley City,49.101176,-122.673733,0.0
31,Langley Township Northwest,48.424523,-123.363029,0.0


In [66]:
for index, row in finalplot.iterrows():

    folium.CircleMarker([row['latitude'], row['longitude']],
                        radius=20,
                        popup=row['Locality'],
                        fill_color="Red", # divvy color
                        control_scale=True,
                       ).add_to(businessmap)
print('Plotted!')

Plotted!


In [67]:
businessmap